# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
#Import your libraries
import numpy as np
import pandas as pd



# Challenge 1 - Import and Describe the Dataset

#### In this challenge we will use the `austin_weather` data. 

#### First, import it into a data frame called `austin`. You can find the data [here](https://drive.google.com/file/d/1dJEvgjTYxd4JCGQ9Zki_irzilkuMFq9p/view?usp=sharing). Please download them and place them in the local data folder

In [ ]:
# Your code here
austin = pd.read_csv('../data/austin_weather.csv')
austin.head()


#### Next, describe the dataset you have loaded: 
- Look at the variables and their types
- Examine the descriptive statistics of the numeric variables 
- Look at the first five rows of all variables to evaluate the categorical variables as well

In [ ]:
# Your code here

austin.dtypes


In [ ]:
# Your code here

austin.describe()
#Paolo: ok

#### Given the information you have learned from examining the dataset, write down three insights about the data in a markdown cell below

#### Insight example:

There are 21 variables in the dataset. 3 of them are numeric and the rest contain some text.<br>

#### Your insights
1. The columns related to temperature are all integers and the remaining values are all objects
2. The column "Date" is in date format and  the column "Events" is in text format
3. The columns related to Sea Pressure and the column related to Precipitation have float values.

#### Let's examine the DewPointAvgF variable by using the `unique()` function to list all unique values in this dataframe.

Describe what you find in a markdown cell below the code. What did you notice? What do you think made Pandas to treat this column as *object* instead of *int64*? 

In [ ]:
# Your code here
austin['DewPointAvgF'].unique()

# Your observation here

There is a value '-' in the column. It is probably a missing value. That is why the Pandas treated this column as object instead of int64.

In [ ]:
#Paolo: ok

The following is a list of columns misrepresented as `object`. Use this list to convert the columns to numeric using the `pandas.to_numeric` function in the next cell. If you encounter errors in converting strings to numeric values, you need to catch those errors and force the conversion by supplying `errors='coerce'` as an argument for `pandas.to_numeric`. Coercing will replace non-convertable elements with `NaN` which represents an undefined numeric value. This makes it possible for us to conveniently handle missing values in subsequent data processing.

*Hint: you may use a loop to change one column at a time but it is more efficient to use `apply`.*

In [ ]:
wrong_type_columns = ['DewPointHighF', 'DewPointAvgF', 'DewPointLowF', 'HumidityHighPercent', 
                      'HumidityAvgPercent', 'HumidityLowPercent', 'SeaLevelPressureHighInches', 
                      'SeaLevelPressureAvgInches' ,'SeaLevelPressureLowInches', 'VisibilityHighMiles',
                      'VisibilityAvgMiles', 'VisibilityLowMiles', 'WindHighMPH', 'WindAvgMPH', 
                      'WindGustMPH', 'PrecipitationSumInches']

In [ ]:
# Your code here
austin[wrong_type_columns] = austin[wrong_type_columns].apply(pd.to_numeric, errors='coerce' )
austin[wrong_type_columns]


#### Check if your code has worked by printing the data types again. You should see only two `object` columns (`Date` and `Events`) now. All other columns should be `int64` or `float64`.

In [ ]:
# Your code here
austin.dtypes
#Paolo: ok

# Challenge 2 - Handle the Missing Data

#### Now that we have fixed the type mismatch, let's address the missing data.

By coercing the columns to numeric, we have created `NaN` for each cell containing characters. We should choose a strategy to address these missing data.

The first step is to examine how many rows contain missing data.

We check how much missing data we have by applying the `.isnull()` function to our dataset. To find the rows with missing data in any of its cells, we apply `.any(axis=1)` to the function. `austin.isnull().any(axis=1)` will return a column containing true if the row contains at least one missing value and false otherwise. Therefore we must subset our dataframe with this column. This will give us all rows with at least one missing value. 

#### In the next cell, identify all rows containing at least one missing value. Assign the dataframes with missing values to a variable called `missing_values`.

In [ ]:
# Your code here

austin['missing_values'] = austin.isnull().any(axis=1)
austin['missing_values']

There are multiple strategies to handle missing data. Below lists the most common ones data scientists use:

* Removing all rows or all columns containing missing data. This is the simplest strategy. It may work in some cases but not others.

* Filling all missing values with a placeholder value. 
    * For categorical data, `0`, `-1`, and `9999` are some commonly used placeholder values. 
    * For continuous data, some may opt to fill all missing data with the mean. This strategy is not optimal since it can increase the fit of the model.

* Filling the values using some algorithm. 

#### In our case, we will use a hybrid approach which is to first remove the data that contain most missing values then fill in the rest of the missing values with the *linear interpolation* algorithm.

#### Next, count the number of rows of `austin` and `missing_values`.

In [ ]:
# Number of rows of austin
len(austin['missing_values'])

In [ ]:
# To count the number of missing values

austin['missing_values'].sum()

# Sum gives us the sum of all the missing values where there is a "True" from the 'missing_values' column

#### Calculate the ratio of missing rows to total rows

In [ ]:
# Method 1:
ratio_missing = austin['missing_values'].sum()/len(austin['missing_values'])*100

ratio_missing
#Paolo: yes!

# Method 2:
# this method gives you ratio of both the values which have 'True' and also the value which have 'False'
# Here normalize is set to True as it gives the relative frequencies of the unique values.
# austin['missing_values'].value_counts(normalize=True)*100

As you can see, there is a large proportion of missing data (over 10%). Perhaps we should evaluate which columns have the most missing data and remove those columns. For the remaining columns, we will perform a linear approximation of the missing data.

We can find the number of missing rows in each column using the `.isna()` function. We then chain the `.sum` function to the `.isna()` function and find the number of missing rows per column

In [ ]:
# Your code here
austin.isna().sum()

#### As you can see from the output, the majority of missing data is in one column called `PrecipitationSumInches`. What's the number of missing values in this column in ratio to its total number of rows?

In [ ]:
# Your code here

(austin['PrecipitationSumInches'].isna().sum())/len(austin) * 100

Over 10% data missing! Therefore, we prefer to remove this column instead of filling its missing values. It doesn't make sense to *guess* its missing values because the estimation will be too 

#### Remove this column from `austin` using the `.drop()` function. Use the `inplace=True` argument.

*Hints:*

* By supplying `inplace=True` to `drop()`, the original dataframe object will be changed in place and the function will return `None`. In contrast, if you don't supply `inplace=True`, which is equivalent to supplying `inplace=False` because `False` is the default value, the original dataframe object will be kept and the function returns a copy of the transformed dataframe object. In the latter case, you'll have to assign the returned object back to your variable.

* Also, since you are dropping a column instead of a row, you'll need to supply `axis=1` to `drop()`.

[Reference for `pandas.DataFrame.drop`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)

In [ ]:
# Your code here 

austin.drop(['PrecipitationSumInches'], axis=1, inplace=True)

# Print `austin` to confirm the column is indeed removed

austin.head()
#Paolo: you can do austin.columns() to check the columns. In this case, since you have many columns,
# if you just do austin.head() you will not be able to visualize all columns in dataframe. Do you see
# the 3 dots after column HumidityLowPercent here below. It says there are other columns
# but it does not let you visualize them. austin.columns() shows all columns
austin.columns

In [ ]:
#Paolo: option with columns
austin.columns

#### Next we will perform linear interpolation of the missing data.

This means that we will use a linear algorithm to estimate the missing data. Linear interpolation assumes that there is a straight line between the points and the missing point will fall on that line. This is a good enough approximation for weather related data. Weather related data is typically a time series. Therefore, we do not want to drop rows from our data if possible. It is prefereable to estimate the missing values rather than remove the rows. However, if you have data from a single point in time, perhaps a better solution would be to remove the rows. 

If you would like to read more about linear interpolation, you can do so [here](https://en.wikipedia.org/wiki/Linear_interpolation).

In the following cell, use the `.interpolate()` function on the entire dataframe. This time pass the `inplace=False` argument to the function and assign the interpolated dataframe to a new variable called `austin_fixed` so that we can compare with `austin`.

In [ ]:
# Your code here

austin_fixed = austin.interpolate()

austin_fixed

#### Check to make sure `austin_fixed` contains no missing data. Also check `austin` - it still contains missing data.

In [ ]:
# Your code here

austin_fixed.isna().sum()

# austin_fixed contains no missing values

In [ ]:
austin.isna().sum()

# austin still has missing values

# Challenge 3 - Processing the `Events` Column

#### Our dataframe contains one true text column - the Events column. We should evaluate this column to determine how to process it.

Use the `value_counts()` function to evaluate the contents of this column

In [ ]:
# Your code here:
austin_fixed['Events'].value_counts()

Reading the values of `Events` and reflecting what those values mean in the context of data, you realize this column indicates what weather events had happened in a particular day.

#### What is the largest number of events happened in a single day? Enter your answer in the next cell.

In [ ]:
# Your answer:

# The largest number of events is a single day is - Fog, Rain, Thunderstorm. It happened on 33 days.
#Paolo:ok

#### We want to transform the string-type `Events` values to the numbers. This will allow us to apply machine learning algorithms easily.

How? We will create a new column for each type of events (i.e. *Rain*, *Snow*, *Fog*, *Thunderstorm*. In each column, we use `1` to indicate if the corresponding event happened in that day and use `0` otherwise.

Below we provide you a list of all event types. Loop the list and create a dummy column with `0` values for each event in `austin_fixed`. To create a new dummy column with `0` values, simply use `austin_fixed[event] = 0`.

In [ ]:
event_list = ['Snow', 'Fog', 'Rain', 'Thunderstorm']

# Your code here

for event in event_list:
    austin_fixed[event] = 0
    

# Print your new dataframe to check whether new columns have been created:

austin_fixed.head()

#### Next, populate the actual values in the dummy columns of  `austin_fixed`.

You will check the *Events* column. If its string value contains `Rain`, then the *Rain* column should be `1`. The same for `Snow`, `Fog`, and `Thunderstorm`.

*Hints:*

* Use [`pandas.Series.str.contains()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html) to create the value series of each new column.

* What if the values you populated are booleans instead of numbers? You can cast the boolean values to numbers by using `.astype(int)`. For instance, `pd.Series([True, True, False]).astype(int)` will return a new series with values of `[1, 1, 0]`.

In [ ]:
# Your code here

event_list = ['Snow', 'Fog', 'Rain', 'Thunderstorm']

# Your code here
 
for event in event_list:
    austin_fixed[event] = austin_fixed['Events'].str.contains(event).astype(int)


#### Print out `austin_fixed` to check if the event columns are populated with the intended values

In [ ]:
# Your code here
austin_fixed.head()

#### If your code worked correctly, now we can drop the `Events` column as we don't need it any more.

In [ ]:
# Your code here

austin_fixed.drop(['Events'], axis = 1, inplace=True)

austin_fixed.head()
#Paolo: same comment as above, option to use austin_fixed.columns()


# Challenge 4 - Processing The `Date` Column

The `Date` column is another non-numeric field in our dataset. A value in that field looks like `'2014-01-06'` which consists of the year, month, and day connected with hyphens. One way to convert the date string to numerical is using a similar approach as we used for `Events`, namely splitting the column into numerical `Year`, `Month`, and `Day` columns. In this challenge we'll show you another way which is to use the Python `datetime` library's `toordinal()` function. Depending on what actual machine learning analysis you will conduct, each approach has its pros and cons. Our goal today is to practice data preparation so we'll skip the discussion here.

Here you can find the [reference](https://docs.python.org/3/library/datetime.html) and [example](https://stackoverflow.com/questions/39846918/convert-date-to-ordinal-python) for `toordinal`. The basic process is to first convert the string to a `datetime` object using `datetime.datetime.strptime`, then convert the `datetime` object to numerical using `toordinal`.

#### In the cell below, convert the `Date` column values from string to numeric values using `toordinal()`.

In [ ]:
# Your code here

import datetime

def convert_to_ordinal(date):
    format_str = '%Y-%m-%d' # Format of the date string
    return datetime.datetime.strptime(date, format_str).toordinal()
    
austin_fixed['Date'] = austin_fixed['Date'].apply(convert_to_ordinal)
    
#Paolo: yes!


#### Print `austin_fixed` to check your `Date` column.

In [ ]:
austin_fixed.head()

# Challenge 5 - Sampling and Holdout Sets

#### Now that we have processed the data for machine learning, we will separate the data to test and training sets.

We first train the model using only the training set. We check our metrics on the training set. We then apply the model to the test set and check our metrics on the test set as well. If the metrics are significantly more optimal on the training set, then we know we have overfit our model. We will need to revise our model to ensure it will be more applicable to data outside the test set.

#### In the next cells we will separate the data into a training set and a test set using the `train_test_split()` function in scikit-learn.

When using `scikit-learn` for machine learning, we first separate the data to predictor and response variables. This is the standard way of passing datasets into a model in `scikit-learn`. The `scikit-learn` will then find out whether the predictors and responses fit the model.

In the next cell, assign the `TempAvgF` column to `y` and the remaining columns to `X`. Your `X` should be a subset of `austin_fixed` containing the following columns: 

```['Date',
 'TempHighF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'Snow',
 'Fog',
 'Rain',
 'Thunderstorm']```
 
 Your `y` should be a subset of `austin_fixed` containing one column `TempAvgF`.

In [ ]:
# Your code here:

y = austin_fixed['TempAvgF']
X = austin_fixed[['TempHighF', 'TempLowF', 'DewPointHighF', 'DewPointAvgF', 'DewPointLowF', 'HumidityHighPercent', 'HumidityAvgPercent', 'HumidityLowPercent', 'SeaLevelPressureHighInches', 'SeaLevelPressureAvgInches', 'SeaLevelPressureLowInches', 'VisibilityHighMiles', 'VisibilityAvgMiles', 'VisibilityLowMiles', 'WindHighMPH', 'WindAvgMPH', 'WindGustMPH', 'Snow', 'Fog', 'Rain', 'Thunderstorm']]

In the next cell, import `train_test_split` from `sklearn.model_selection`

In [ ]:
#Your code here:

from sklearn.model_selection import train_test_split

Now that we have split the data to predictor and response variables and imported the `train_test_split()` function, split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`. 80% of the data should be in the training set and 20% in the test set. `train_test_split()` reference can be accessed [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).


Enter your code in the cell below:

In [ ]:
#Your code here:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#### Congratulations! Now you have finished the preparation of the dataset!

# Bonus Challenge 1

#### While the above is the common practice to prepare most datasets, when it comes to time series data, we sometimes do not want to randomly select rows from our dataset.

This is because many time series algorithms rely on observations having equal time distances between them. In such cases, we typically select the majority of rows as the test data and the last few rows as the training data. We don't use `train_test_split()` to select the train/test data because it returns random selections.

In the following cell, compute the number of rows that account for 80% of our data and round it to the next integer. Assign this number to `ts_rows`.

In [ ]:
# Your code here:
len(austin_fixed)*0.8

# Here the result is 1055.2 and I will round it to the next integer which is 1056

ts_rows = 1056
#Paolo :ok

Assign the first `ts_rows` rows of `X` to `X_ts_train` and the remaining rows to `X_ts_test`.

In [ ]:
# Your code here:

X_ts_train = austin_fixed.iloc [0:1057]
X_ts_test = austin_fixed.iloc [1057:]
#Paolo :yes! consider using ts_rows instead of hardcoding the number.So
# X_ts_train = austin_fixed.iloc [0:ts_rows+1] instead of X_ts_train = austin_fixed.iloc [0:1057]
# That way if you want to change the pecentage you change it only once setting a different number
# ofr ts_rows
# Same comments for cells below

Assign the first `ts_rows` rows of `y` to `y_ts_train` and the remaining rows to `y_ts_test`.

In [ ]:
# Your code here:
y_ts_train = austin_fixed.iloc [0:1057]
y_ts_test = austin_fixed.iloc [1057:]
#Paolo: yes